In [52]:
import argparse
import os
import shutil
import chromadb
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from chromadb import EmbeddingFunction
from langchain.vectorstores.chroma import Chroma

from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
from chromadb.utils import embedding_functions
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

DATA_PATH = "C:/Users/linus/Universitaet/LAI/Praktikum/Textgrundlage/Archiv/Archiv2/3"

In [54]:
client = chromadb.PersistentClient(path= DATA_PATH)
CHROMA_PATH = "C:/Users/linus/chroma_db"

In [56]:
loader = PyPDFDirectoryLoader(DATA_PATH)
documents = loader.load()
embedding_function = OllamaEmbeddings(model="llama3.2:latest")
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
chunks = splitter.split_documents(documents)
#chunks = split_documents(documents)


Anzahl der Chunks, die hinzugefügt werden sollen: 682


In [37]:
existing_documents = db.get(include=['metadatas'])
print(f"Metadaten der gespeicherten Dokumente: {existing_documents['metadatas']}")
print(f"Anzahl der gespeicherten Dokumente: {len(existing_documents['metadatas'])}")

Metadaten der gespeicherten Dokumente: [{'page': 0, 'source': 'C:\\Users\\linus\\Universitaet\\LAI\\Praktikum\\Textgrundlage\\Archiv\\Archiv2\\2\\12_SRLAM0252_0003.pdf'}, {'page': 0, 'source': 'C:\\Users\\linus\\Universitaet\\LAI\\Praktikum\\Textgrundlage\\Archiv\\Archiv2\\2\\12_SRLAM0252_0003.pdf'}, {'page': 0, 'source': 'C:\\Users\\linus\\Universitaet\\LAI\\Praktikum\\Textgrundlage\\Archiv\\Archiv2\\2\\12_SRLAM0252_0003.pdf'}, {'page': 1, 'source': 'C:\\Users\\linus\\Universitaet\\LAI\\Praktikum\\Textgrundlage\\Archiv\\Archiv2\\2\\12_SRLAM0252_0003.pdf'}, {'page': 0, 'source': 'C:\\Users\\linus\\Universitaet\\LAI\\Praktikum\\Textgrundlage\\Archiv\\Archiv2\\2\\13_SRLAM0252_0004.pdf'}, {'page': 0, 'source': 'C:\\Users\\linus\\Universitaet\\LAI\\Praktikum\\Textgrundlage\\Archiv\\Archiv2\\2\\13_SRLAM0252_0004.pdf'}, {'page': 0, 'source': 'C:\\Users\\linus\\Universitaet\\LAI\\Praktikum\\Textgrundlage\\Archiv\\Archiv2\\2\\13_SRLAM0252_0004.pdf'}, {'page': 0, 'source': 'C:\\Users\\linus\\Un

In [58]:
import time

def add_to_chroma(chunks):
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
    print(f"Anzahl der Chunks, die hinzugefügt werden sollen: {len(chunks)}")

    start_time = time.time()
    for i, chunk in enumerate(chunks):
        db.add_documents([chunk])  # Einen Chunk pro Schritt hinzufügen
        if i % 10 == 0:  # Fortschritt alle 10 Chunks melden
            elapsed_time = time.time() - start_time
            print(f"Chunks hinzugefügt: {i+1}/{len(chunks)} - Verstrichene Zeit: {elapsed_time:.2f} Sekunden")
    db.persist()
    total_time = time.time() - start_time
    print(f"✨ Hinzufügen abgeschlossen. Gesamtdauer: {total_time:.2f} Sekunden")

In [60]:
add_to_chroma(chunks)

Anzahl der Chunks, die hinzugefügt werden sollen: 656
Chunks hinzugefügt: 1/656 - Verstrichene Zeit: 28.95 Sekunden
Chunks hinzugefügt: 11/656 - Verstrichene Zeit: 258.98 Sekunden
Chunks hinzugefügt: 21/656 - Verstrichene Zeit: 484.96 Sekunden
Chunks hinzugefügt: 31/656 - Verstrichene Zeit: 728.04 Sekunden
Chunks hinzugefügt: 41/656 - Verstrichene Zeit: 941.76 Sekunden
Chunks hinzugefügt: 51/656 - Verstrichene Zeit: 1118.64 Sekunden
Chunks hinzugefügt: 61/656 - Verstrichene Zeit: 1308.64 Sekunden
Chunks hinzugefügt: 71/656 - Verstrichene Zeit: 1559.15 Sekunden
Chunks hinzugefügt: 81/656 - Verstrichene Zeit: 1885.41 Sekunden
Chunks hinzugefügt: 91/656 - Verstrichene Zeit: 2116.93 Sekunden
Chunks hinzugefügt: 101/656 - Verstrichene Zeit: 2316.35 Sekunden
Chunks hinzugefügt: 111/656 - Verstrichene Zeit: 2571.18 Sekunden
Chunks hinzugefügt: 121/656 - Verstrichene Zeit: 2842.96 Sekunden
Chunks hinzugefügt: 131/656 - Verstrichene Zeit: 3096.57 Sekunden
Chunks hinzugefügt: 141/656 - Verstrich

In [62]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [64]:
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
##def query_rag(query_text: str):
    ##embedding_function = default_ef
    #default_ef = embedding_functions.DefaultEmbeddingFunction()
   ## db = Chroma(
        #persist_directory=CHROMA_PATH,
        #embedding_function=embedding_function
        #default_ef=default_ef
    #)
def query_rag(query_text: str):
    # Stelle sicher, dass das Modell eine embed_query-Methode hat
    embedding_function = OllamaEmbeddings(model="llama3.2:latest")
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=embedding_function
    )
    
    results = db.similarity_search_with_score(query_text, k=5)
    
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    
    model = OllamaLLM(model="llama3.2:latest")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [66]:
query_rag("What can you tell me about the documents provided?")

Response: Based on the provided context, I can tell you that:

* The documents appear to be related to geological or oil exploration data, as they contain measurements of pressure and temperature for various wells (e.g. "productl", "crude", "liquid", etc.) at different depths.
* The data is presented in a tabular format with multiple columns, indicating that it may have been generated by an automated system or database.
* There are also charts and graphs accompanying the text, specifically Figure 3-1 which shows wind speed distribution in 1987.
* The documents seem to be written in a technical or scientific tone, possibly for an academic or professional audience.

Without more information, it's difficult to provide further context or insights about the specific topic or purpose of these documents.
Sources: [None, None, None, None, None]


'Based on the provided context, I can tell you that:\n\n* The documents appear to be related to geological or oil exploration data, as they contain measurements of pressure and temperature for various wells (e.g. "productl", "crude", "liquid", etc.) at different depths.\n* The data is presented in a tabular format with multiple columns, indicating that it may have been generated by an automated system or database.\n* There are also charts and graphs accompanying the text, specifically Figure 3-1 which shows wind speed distribution in 1987.\n* The documents seem to be written in a technical or scientific tone, possibly for an academic or professional audience.\n\nWithout more information, it\'s difficult to provide further context or insights about the specific topic or purpose of these documents.'